In [1]:
# schema.drop(True)
# TrialProcessed.drop()
# PlotTable.drop()

In [2]:
## package setup and import



import numpy as np
import datajoint as dj
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from scipy.io import loadmat
import datetime


# path = '/Users/ashwi/Documents/GitHub/datapipeline'
# files = os.listdir(path)
folder = os.getcwd()


dj.config['stores'] = {
    'raw_data':{
        'protocol':'file',
        'location': os.path.join(folder,'raw_data')
    }
}

## datajoint setup/import for database management
dj.__version__

dj.config['database.host'] = '127.0.0.1'
dj.config['database.port'] = 3306
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'simple'
dj.config['fetch_format'] = 'array'
dj.config['enable_python_native_blobs'] = True 


connection = dj.conn()
connection

dj.config.save_global()

schema = dj.schema('behavioral')
schema.drop(True)
schema = dj.schema('behavioral')

schema.spawn_missing_classes()

dj.ERD(schema)

Connecting root@127.0.0.1:3306


FileNotFoundError: [WinError 2] "dot" not found in path.

In [3]:
## create table with filenames and file locations and corresponding date and mouse ID
@schema 
class FileTable(dj.Lookup):
    definition = """
    filename: varchar(300)
    ---
    file : varchar(300)
    mouse_id: varchar(100)
    date: date
    """
    
    def loadfolder(folder):
        datafiles = []
        filenames = []
        for file in glob.glob(folder):
            filename = (file[file.rfind('JB'):])
            datafile = (file)
            mouse_id = filename.split('_')[0]
            datestr = filename.split('_')[2]
            date_obj = datetime.datetime.strptime(datestr, '%Y%m%d')
            date = date_obj.date()
            try:
                test = loadmat(file)['SessionData']['RawEvents']
#                 datafile = (file)
#                 mouse_id = filename.split('_')[0]
#                 datestr = filename.split('_')[2]
#                 date_obj = datetime.datetime.strptime(datestr, '%Y%m%d')
#                 date = date_obj.date()
                FileTable.insert1({'file' : datafile,'filename' : filename, 'mouse_id' : mouse_id, 'date' : date},skip_duplicates = True)
#                 print(filename + ': loaded')
            except:
                print(filename + ': incomplete')
            

        
            
## Load all data files in data folder
# C:\Users\ashwi\Documents\testdata\JB*.mat
# FileTable.loadfolder(r"C:\Users\ashwi\Dropbox\BpodInfoseek\Data\JB*\TestInfoseek\Session Data\JB*.mat")
# FileTable.loadfolder(r"C:\Users\ashwi\Dropbox\BpodInfoseek\Data\JB*\*Infoseek\Session Data\JB*.mat")
FileTable.loadfolder(r"D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB*\*Infoseek\Session Data\JB*.mat")
FileTable()

JB366_Infoseek_20201014_104156.mat: incomplete
JB368_Infoseek_20201007_131457.mat: incomplete
JB368_Infoseek_20201012_162037.mat: incomplete
JB368_Infoseek_20201016_140829.mat: incomplete
JB368_TestInfoseek_20201012_135518.mat: incomplete
JB368_TestInfoseek_20201012_135552.mat: incomplete
JB369_Infoseek_20201013_142758.mat: incomplete


filename,file,mouse_id,date
JB366_Infoseek_20201013_102926.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB366\Infoseek\Session Data\JB366_Infoseek_20201013_102926.mat,JB366,2020-10-13
JB366_Infoseek_20201014_103353.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB366\Infoseek\Session Data\JB366_Infoseek_20201014_103353.mat,JB366,2020-10-14
JB366_Infoseek_20201014_104211.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB366\Infoseek\Session Data\JB366_Infoseek_20201014_104211.mat,JB366,2020-10-14
JB366_Infoseek_20201015_101140.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB366\Infoseek\Session Data\JB366_Infoseek_20201015_101140.mat,JB366,2020-10-15
JB366_Infoseek_20201016_102403.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB366\Infoseek\Session Data\JB366_Infoseek_20201016_102403.mat,JB366,2020-10-16
JB366_Infoseek_20201016_104533.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB366\Infoseek\Session Data\JB366_Infoseek_20201016_104533.mat,JB366,2020-10-16
JB366_TestInfoseek_20201012_103513.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB366\TestInfoseek\Session Data\JB366_TestInfoseek_20201012_103513.mat,JB366,2020-10-12
JB367_Infoseek_20201013_112258.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB367\Infoseek\Session Data\JB367_Infoseek_20201013_112258.mat,JB367,2020-10-13
JB367_Infoseek_20201014_112858.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB367\Infoseek\Session Data\JB367_Infoseek_20201014_112858.mat,JB367,2020-10-14
JB367_Infoseek_20201015_105831.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB367\Infoseek\Session Data\JB367_Infoseek_20201015_105831.mat,JB367,2020-10-15


In [4]:
# datafiles = []
# filenames = []
# # C:\Users\ashwi\Documents\testdata
# # C:\Users\ashwi\Dropbox\BpodInfoseek\Data\JB*\TestInfoseek\Session Data\*.mat
# for file in glob.glob(r"C:\Users\ashwi\Dropbox\BpodInfoseek\Data\JB*\TestInfoseek\Session Data\*.mat"):
#         filenames.append(file[file.find('JB',45):])
#         datafiles.append(file)

In [5]:
## manual entry mouse ID table

@schema
class MouseAttributes(dj.Manual): ## Manual entry table for mouse identification/attribute labelling
    definition = """
    mouse_id : varchar(100) 
    ---
    sex = 'U': enum('F','M','U')
    imaging = null : int
    real_infoseek_data = null : int
    real_infoseek_no_imaging = null : int
    mean_preference = null : varchar(1000)
    full_values = null : int
    one_vs_four = null : int
    exp_type = null : varchar(1000)
    reward_type = null : varchar(1000)
    imaging = null : int
    imaging_comments = null : varchar(2000)
    """
    
    def newMouse():
        mice = []
        mice = np.unique(FileTable.fetch()['mouse_id'])
        for i in np.arange(len(mice)):
            MouseAttributes.insert1({'mouse_id':mice[i]},skip_duplicates=True)
MouseAttributes.newMouse()
MouseAttributes()
# format for entry of new mouse with new attributes, load later on will create new mouse if not previously entered but will be missing experimental params
# MouseAttributes.insert1({'mouse_id' : 'FakeSubject', 'mean_preference': 'test entry','exp_type': '3 port long'},skip_duplicates=True)


mouse_id,sex,imaging,real_infoseek_data,real_infoseek_no_imaging,mean_preference,full_values,one_vs_four,exp_type,reward_type,imaging_comments
JB366,U,None,None,None,None,None,None,None,None,None
JB367,U,None,None,None,None,None,None,None,None,None
JB368,U,None,None,None,None,None,None,None,None,None
JB369,U,None,None,None,None,None,None,None,None,None


In [6]:
# datafiles associated with specific testing dates and times with what associated file name

@schema
class TestingSession(dj.Imported): ## Manual entry table for each recording session -- want to automate (imported data table - import table characteristics from recording files)
    definition = """
    -> MouseAttributes
    date : date
    time : varchar(100)
    -> FileTable
    ---
    datafile : varchar(300)
    filename : varchar(200)
    comments = null : varchar(1000)
    """



    def make(self,key):
        datafiles = ((FileTable & key).fetch()['file'])
        filenames = ((FileTable & key).fetch()['filename'])
        for i in np.arange(len(datafiles)):
            datafile = datafiles[i]
            filename = filenames[i]
            mouse_id = filename.split('_')[0]
            date_str = filename.split('_')[2]
            date_obj = datetime.datetime.strptime(date_str, '%Y%m%d')
            date = date_obj.date()
            time_str = (filename.split('_')[3]).split('.mat')[0]
            time_obj = datetime.datetime.strptime(time_str, '%H%M%S')
            time = time_obj.time()
            key['mouse_id'] = mouse_id
            key['date'] = date
            key['time'] = time
            key['datafile'] = datafile
            key['filename'] = filename
            self.insert1(key)
        
        
TestingSession.populate()
TestingSession()

mouse_id,date,time,filename,datafile,comments
JB366,2020-10-12,10:35:13,JB366_TestInfoseek_20201012_103513.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB366\TestInfoseek\Session Data\JB366_TestInfoseek_20201012_103513.mat,None
JB366,2020-10-13,10:29:26,JB366_Infoseek_20201013_102926.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB366\Infoseek\Session Data\JB366_Infoseek_20201013_102926.mat,None
JB366,2020-10-14,10:33:53,JB366_Infoseek_20201014_103353.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB366\Infoseek\Session Data\JB366_Infoseek_20201014_103353.mat,None
JB366,2020-10-14,10:42:11,JB366_Infoseek_20201014_104211.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB366\Infoseek\Session Data\JB366_Infoseek_20201014_104211.mat,None
JB366,2020-10-15,10:11:40,JB366_Infoseek_20201015_101140.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB366\Infoseek\Session Data\JB366_Infoseek_20201015_101140.mat,None
JB366,2020-10-16,10:24:03,JB366_Infoseek_20201016_102403.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB366\Infoseek\Session Data\JB366_Infoseek_20201016_102403.mat,None
JB366,2020-10-16,10:45:33,JB366_Infoseek_20201016_104533.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB366\Infoseek\Session Data\JB366_Infoseek_20201016_104533.mat,None
JB367,2020-10-12,11:30:19,JB367_TestInfoseek_20201012_113019.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB367\TestInfoseek\Session Data\JB367_TestInfoseek_20201012_113019.mat,None
JB367,2020-10-13,11:22:58,JB367_Infoseek_20201013_112258.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB367\Infoseek\Session Data\JB367_Infoseek_20201013_112258.mat,None
JB367,2020-10-14,11:28:58,JB367_Infoseek_20201014_112858.mat,D:\Dropbox\BpodInfoseek\Data\PythonTest2\JB367\Infoseek\Session Data\JB367_Infoseek_20201014_112858.mat,None


In [7]:
# # enter data into manual entry tables

# def newEntry(datafile):
#     filename = datafile[datafile.find('JB',20):]
#     mouse_id = filename.split('_')[0]
#     MouseAttributes.insert1({'mouse_id': mouse_id},skip_duplicates=True)
#     TestingSession().insertSession(datafile)



    
# for i in datafiles: 
#     try:
#         newEntry(i)
#         print(i + ': loaded')
#     except ValueError:
#         print(i + ': incomplete file')



# # for file in os.listdir(r"C:\Users\ashwi\Dropbox\BpodInfoseek\Data\JB366\CenterTraining\Session Data"):
# #     if file.startswith("JB"):
# #         print(os.listdir(r"C:\Users\ashwi\Dropbox\BpodInfoseek\Data\JB366\CenterTraining\Session Data"))
# #         newEntry(os.path.join(r"C:\Users\ashwi\Dropbox\BpodInfoseek\Data\JB366\CenterTraining\Session Data",file),str())
# # dj.ERD(schema)


In [8]:
@schema
class DateTable(dj.Imported):
    definition = """
    -> MouseAttributes
    date : date
    ---
    """
    
    def make(self,key):
        mouse_id = (FileTable & key).fetch()['mouse_id']
        date = (FileTable & key).fetch()['date']
        for i in np.arange(len((FileTable & key).fetch()['date'])):
            key['date'] = date[i]
            self.insert1(key,skip_duplicates=True)
        
DateTable.populate()
DateTable()

mouse_id,date
JB366,2020-10-12
JB366,2020-10-13
JB366,2020-10-14
JB366,2020-10-15
JB366,2020-10-16
JB367,2020-10-12
JB367,2020-10-13
JB367,2020-10-14
JB367,2020-10-15
JB367,2020-10-16


In [9]:
# list raw data and trial settings 

@schema
class SessionData(dj.Imported):
    definition = """
    -> MouseAttributes
    -> TestingSession
    ---
    raw_data : longblob

    """
    
    def make(self,key):
        data = loadmat((TestingSession & key).fetch1()['datafile'])
#         settings = pd.DataFrame(data['SessionData']['TrialSettings'][0][0][0])
#         settingsdict = data['SessionData']['TrialSettings'][0][0][0]
        key['raw_data'] = pd.DataFrame(data['SessionData'][0])
        self.insert1(key)

SessionData.populate()
SessionData()

mouse_id,date,time,filename,raw_data
JB366,2020-10-12,10:35:13,JB366_TestInfoseek_20201012_103513.mat,=BLOB=
JB366,2020-10-13,10:29:26,JB366_Infoseek_20201013_102926.mat,=BLOB=
JB366,2020-10-14,10:33:53,JB366_Infoseek_20201014_103353.mat,=BLOB=
JB366,2020-10-14,10:42:11,JB366_Infoseek_20201014_104211.mat,=BLOB=
JB366,2020-10-15,10:11:40,JB366_Infoseek_20201015_101140.mat,=BLOB=
JB366,2020-10-16,10:24:03,JB366_Infoseek_20201016_102403.mat,=BLOB=
JB366,2020-10-16,10:45:33,JB366_Infoseek_20201016_104533.mat,=BLOB=
JB367,2020-10-12,11:30:19,JB367_TestInfoseek_20201012_113019.mat,=BLOB=
JB367,2020-10-13,11:22:58,JB367_Infoseek_20201013_112258.mat,=BLOB=
JB367,2020-10-14,11:28:58,JB367_Infoseek_20201014_112858.mat,=BLOB=


In [10]:
# create table that compiles water and number of trials per experimental session

@schema
class SessionWater(dj.Computed):
    definition = """
    -> TestingSession
    ---
    water_vol_ul : int
    drop_size : int
    num_trials : int
    """
    
    def make(self,key):
        dropVol = 4 # mL
        data = ((SessionData & key).fetch1()['raw_data'])
        trialEvents = data['RawEvents'][0][0][0][0][0]
        numDrops = []
        numTrials = len(trialEvents)
        for i in np.arange(len(trialEvents)):
            try:
                dropCount = len(trialEvents[i][0][0][1]['GlobalTimer3_End'][0][0][0])
            except ValueError:
                try:
                    dropCount = len(trialEvents[i][0][0][1]['GlobalTimer4_End'][0][0][0])
                except ValueError:
                    dropCount = 0

            numDrops.append(dropCount)

        totalDrops = sum(numDrops)
        
        key['water_vol_ul'] = totalDrops*dropVol
        key['drop_size'] = dropVol
        key['num_trials'] = numTrials

        self.insert1(key)
        
SessionWater.populate()
SessionWater()

mouse_id,date,time,filename,water_vol_ul,drop_size,num_trials
JB366,2020-10-12,10:35:13,JB366_TestInfoseek_20201012_103513.mat,208,4,92
JB366,2020-10-13,10:29:26,JB366_Infoseek_20201013_102926.mat,80,4,67
JB366,2020-10-14,10:33:53,JB366_Infoseek_20201014_103353.mat,0,4,6
JB366,2020-10-14,10:42:11,JB366_Infoseek_20201014_104211.mat,320,4,79
JB366,2020-10-15,10:11:40,JB366_Infoseek_20201015_101140.mat,220,4,71
JB366,2020-10-16,10:24:03,JB366_Infoseek_20201016_102403.mat,156,4,52
JB366,2020-10-16,10:45:33,JB366_Infoseek_20201016_104533.mat,600,4,124
JB367,2020-10-12,11:30:19,JB367_TestInfoseek_20201012_113019.mat,172,4,79
JB367,2020-10-13,11:22:58,JB367_Infoseek_20201013_112258.mat,20,4,54
JB367,2020-10-14,11:28:58,JB367_Infoseek_20201014_112858.mat,592,4,120


In [11]:
# table with day summary of total water given through experimental trials for individual mice each day

@schema
class DaySummaries(dj.Imported):
    definition = """
    -> DateTable
    ---
    total_water_ul : int
    """

    
    def make(self,key):
        mouseID = (DateTable & key).fetch()['mouse_id']
        dates = (DateTable & key).fetch()['date']
        for i in mouseID:
            for j in dates:
                dayWater = sum((SessionWater & 'mouse_id = ' + str('"' + i + '"') & 'date = ' + str('"' + str(j) + '"')).fetch()['water_vol_ul'])
                key['total_water_ul'] = dayWater

        self.insert1(key)    

DaySummaries.populate()
DaySummaries()

mouse_id,date,total_water_ul
JB366,2020-10-12,208
JB366,2020-10-13,80
JB366,2020-10-14,320
JB366,2020-10-15,220
JB366,2020-10-16,756
JB367,2020-10-12,172
JB367,2020-10-13,20
JB367,2020-10-14,592
JB367,2020-10-15,668
JB367,2020-10-16,1020


In [12]:
# @schema
# class OutcomesPlot(dj.Computed):
#     definition = """
#     -> SessionData
#     ---
#     num_trials : int
#     outcomes : longblob
#     """
    
#     def make(self,key):
#         data = ((SessionData & key).fetch1()['raw_data'])
#         outcomes = pd.DataFrame(data['Outcomes'][0][0])
#         numTrials = len(outcomes)
#         key['num_trials'] = numTrials
#         key['outcomes'] = outcomes
#         self.insert1(key)

# OutcomesPlot.populate()
# OutcomesPlot()

In [13]:
@schema 
class SessionProcessed(dj.Imported):
    definition = """
    -> SessionData
    ---
    trial_types : longblob
    outcomes : longblob
    orig_trial_types : longblob
    reward_types : longblob
    n_events : int
    event_names : longblob
    info : longblob
    n_trials : int
    raw_events : longblob
    session_states : longblob
    session_events : longblob
    raw_data : longblob
    trial_start_timestamps : longblob
    trial_end_timestamps : longblob
    settings_file : longblob
    notes : longblob
    marker_codes : longblob
    session_trial_settings : longblob
    session_all_trial_types : longblob
    """
    
    
    def make(self,key):
        data = ((SessionData & key).fetch1()['raw_data'])
        
        trial_types = pd.DataFrame(data['TrialTypes'][0][0])
        outcomes = pd.DataFrame(data['Outcomes'][0][0])
        orig_trial_types = pd.DataFrame(data['OrigTrialTypes'][0])
        try:
            reward_types = pd.DataFrame(data['RewardTypes'][0])
        except KeyError:
            try:
                reward_types = pd.DataFrame(data['OrigRewardTypes'][0])
            except KeyError:
                print('RewardTypes Not Found')
        try:
            n_events = (data['nEvents'][0][0][0])
        except KeyError:
            n_events = 0
        event_names = pd.DataFrame(data['EventNames'][0][0])
        info = pd.DataFrame(data['Info'][0][0])
        n_trials = (data['nTrials'][0][0][0])
        raw_events = pd.DataFrame(data['RawEvents'][0][0][0][0][0])
        raw_data = pd.DataFrame(data['RawData'][0][0])
        trial_start_timestamp = pd.DataFrame(data['TrialStartTimestamp'][0][0])
        trial_end_timestamp = pd.DataFrame(data['TrialEndTimestamp'][0][0])
        settings_file = pd.DataFrame(data['SettingsFile'][0][0])
        notes = pd.DataFrame(data['Notes'][0][0])
        marker_codes = pd.DataFrame(data['MarkerCodes'][0][0])
        session_trial_settings = pd.DataFrame(data['TrialSettings'][0][0])
        try:
            session_trial_types = pd.DataFrame(data['AllTrialTypes'][0][0])
        except KeyError:
            session_trial_types = None
        
        states = []
        events = []
        for i in np.arange(len((SessionData & key).fetch1()['raw_data']['RawEvents'][0][0][0][0][0])):
            states.append((SessionData & key).fetch1()['raw_data']['RawEvents'][0][0][0][0][0][i]['States'])
            events.append((SessionData & key).fetch1()['raw_data']['RawEvents'][0][0][0][0][0][i]['Events'])
        session_states = pd.DataFrame(states)
        session_events = pd.DataFrame(events)
        
        key['trial_types'] = trial_types
        key['outcomes'] = outcomes
        key['orig_trial_types'] = orig_trial_types
        key['reward_types'] = reward_types
        try:
            key['n_events'] = n_events
        except: 
            key['n_events'] = None
        key['event_names'] = event_names
        key['info'] = info
        key['n_trials'] = n_trials
        key['raw_events'] = raw_events
        key['session_states'] = session_states
        key['session_events'] = session_events
        key['raw_data'] = raw_data
        key['trial_start_timestamps'] = trial_start_timestamp
        key['trial_end_timestamps'] = trial_end_timestamp
        key['settings_file'] = settings_file
        key['notes'] = notes
        key['marker_codes'] = marker_codes
        key['session_trial_settings'] = session_trial_settings
        key['session_all_trial_types'] = session_trial_types
        
        self.insert1(key)

SessionProcessed.populate()
SessionProcessed()

NameError: name 'null' is not defined

In [ ]:
@schema
class TrialData(dj.Imported):
    definition = """
    -> SessionProcessed
    trial_num : int
    ---
    trial_type : int
    outcome : int
    trial_states : longblob
    trial_events : longblob
    trial_start_timestamp : float
    trial_end_timestamp : float
    original_state_names_by_number : longblob
    original_state_data : longblob
    original_event_data : longblob
    original_state_timestamps : longblob
    original_event_timestamps : longblob
    state_machine_error_codes : longblob
    trial_settings : longblob
    all_trial_types : longblob
    """
    
    def make(self,key):
        data = ((SessionProcessed & key).fetch1()['raw_data'])
        for i in np.arange((SessionProcessed & key).fetch1()['n_trials']):
            key['trial_num'] = i
            key['trial_type'] = ((SessionProcessed & key).fetch1()['trial_types'][0][i])
            key['outcome'] = ((SessionProcessed & key).fetch1()['outcomes'][0][i])
            key['trial_states'] = pd.DataFrame((SessionProcessed & key).fetch1()['session_states']).iloc[i][0][0][0][0]
            key['trial_events'] = pd.DataFrame((SessionProcessed & key).fetch1()['session_events']).iloc[i][0][0][0][0]
            key['trial_start_timestamp'] = ((SessionProcessed & key).fetch1()['trial_start_timestamps'])[0][i]
            key['trial_end_timestamp'] = ((SessionProcessed & key).fetch1()['trial_end_timestamps'])[0][i]            
            key['original_state_names_by_number'] = pd.DataFrame(data['OriginalStateNamesByNumber'][0][0][i])
            key['original_state_data'] = pd.DataFrame(data['OriginalStateData'][0][0][i])
            key['original_event_data'] = pd.DataFrame(data['OriginalEventData'][0][0][i])
            key['original_state_timestamps'] = pd.DataFrame(data['OriginalStateTimestamps'][0][0][i])
            key['original_event_timestamps'] = pd.DataFrame(data['OriginalEventTimestamps'][0][0][i])
            key['state_machine_error_codes'] = pd.DataFrame(data['StateMachineErrorCodes'][0][0][i])
            key['trial_settings'] = pd.DataFrame((SessionProcessed & key).fetch1()['session_trial_settings'].iloc[i][0][0])
            try:
                key['all_trial_types'] = pd.DataFrame((SessionProcessed & key).fetch1()['session_all_trial_types'].iloc[i][0])
            except:
                key['all_trial_types'] = pd.DataFrame([0])
            self.insert1(key)

TrialData.populate()
TrialData()

In [ ]:
@schema
class TrialStates(dj.Imported):
    definition = """
    -> TrialData
    ---
    start_trial : blob
    wait_for_center : blob
    center_delay : blob
    center_odor : blob
    center_post_odor_delay : blob
    go_cue : blob
    response : blob
    grace_period : blob
    wait_for_odor_l : blob
    wait_for_odor_r : blob
    odor_left : blob
    odor_right : blob
    reward_delay_left : blob
    reward_delay_right : blob
    left_port_check : blob
    left_not_present : blob
    right_port_check : blob
    right_not_present : blob
    left_big_reward : blob
    left_small_reward : blob
    right_big_reward : blob
    right_small_reward : blob
    outcome_delivery : blob
    incorrect_left : blob
    incorrect_right : blob
    no_choice : blob
    incorrect : blob
    timeout_odor : blob
    timeout_reward_delay : blob
    timeout_outcome : blob
    intertrial_interval : blob
    """    

    
    def make(self,key):
        trials = (TrialData & key).fetch1()['trial_states']
        start_trial = trials['StartTrial'][0][0]
        wait_for_center = trials['WaitForCenter'][0][0]
        center_delay = trials['CenterDelay'][0][0]
        center_odor = trials['CenterOdor'][0][0]
        center_post_odor_delay = trials['CenterPostOdorDelay'][0][0]
        go_cue = trials['GoCue'][0][0]
        response = trials['Response'][0][0]
        try:
            grace_period = trials['GracePeriod'][0][0]
        except:
            pass
        wait_for_odor_l = trials['WaitForOdorLeft'][0][0]
        wait_for_odor_r = trials['WaitForOdorRight'][0][0]
        odor_left = trials['OdorLeft'][0][0]
        odor_right = trials['OdorRight'][0][0]
        reward_delay_left = trials['RewardDelayLeft'][0][0]
        reward_delay_right = trials['RewardDelayRight'][0][0]
        left_port_check = trials['LeftPortCheck'][0][0]
        right_port_check = trials['RightPortCheck'][0][0]
        left_not_present = trials['LeftNotPresent'][0][0]
        right_not_present = trials['RightNotPresent'][0][0]
        left_big_reward = trials['LeftBigReward'][0][0]
        left_small_reward = trials['LeftSmallReward'][0][0]
        right_big_reward = trials['RightBigReward'][0][0]
        right_small_reward = trials['RightSmallReward'][0][0]
        outcome_delivery = trials['OutcomeDelivery'][0][0]
        incorrect_left = trials['IncorrectLeft'][0][0]
        incorrect_right = trials['IncorrectRight'][0][0]
        no_choice = trials['NoChoice'][0][0]
        incorrect = trials['Incorrect'][0][0]
        timeout_odor = trials['TimeoutOdor'][0][0]
        timeout_reward_delay = trials['TimeoutRewardDelay'][0][0]
        timeout_outcome = trials['TimeoutOutcome'][0][0]
        intertrial_interval = trials['InterTrialInterval'][0][0]
#         for i in np.arange(len(trials)):
#                 trialstate = trials[i][0][0][0][0]
#                 start_trial[i] = (trialstate['StartTrial'])
#                 wait_for_center[i] = (trialstate['WaitForCenter'])
#                 center_delay[i] = (trialstate['CenterDelay'])
#                 center_odor[i] = (trialstate['CenterOdor'])
#                 center_post_odor_delay[i] = (trialstate['CenterPostOdorDelay'])
#                 go_cue[i] = (trialstate['GoCue'])
#                 response[i] = (trialstate['Response'])
#                 wait_for_odor_l[i] = (trialstate['WaitForOdorLeft'])
#                 wait_for_odor_r[i] = (trialstate['WaitForOdorRight'])
#                 odor_left[i] = (trialstate['OdorLeft'])
#                 odor_right[i] = (trialstate['OdorRight'])
#                 reward_delay_left[i] = (trialstate['RewardDelayLeft'])
#                 reward_delay_right[i] = (trialstate['RewardDelayRight'])
#                 left_port_check[i] = (trialstate['LeftPortCheck'])
#         #             right_port_check[i] = pd.DataFrame(trialstate['RightPortCheck'])
#                 left_not_present[i] = (trialstate['LeftNotPresent'])
#                 right_not_present[i] = (trialstate['RightNotPresent'])
#                 left_big_reward[i] = (trialstate['LeftBigReward'])
#                 left_small_reward[i] = (trialstate['LeftSmallReward'])
#                 right_big_reward[i] = (trialstate['RightBigReward'])
#                 right_small_reward[i] = (trialstate['RightSmallReward'])
#                 outcome_delivery[i] = (trialstate['OutcomeDelivery'])
#                 incorrect_left[i] = (trialstate['IncorrectLeft'])
#                 incorrect_right[i] = (trialstate['IncorrectRight'])
#                 no_choice[i] = (trialstate['NoChoice'])
#                 incorrect[i] = (trialstate['Incorrect'])
#                 timeout_odor[i] = (trialstate['TimeoutOdor'])
#                 timeout_reward_delay[i] = (trialstate['TimeoutRewardDelay'])
#                 timeout_outcome[i]  = (trialstate['TimeoutOutcome'])
#                 intertrial_interval[i] = (trialstate['InterTrialInterval'])
        
            
            
            
        key['start_trial'] = pd.DataFrame(start_trial)
        key['wait_for_center'] = pd.DataFrame(wait_for_center)
        key['center_delay'] = pd.DataFrame(center_delay)
        key['center_odor'] = pd.DataFrame(center_odor)
        key['center_post_odor_delay'] = pd.DataFrame(center_post_odor_delay)
        key['go_cue'] = pd.DataFrame(go_cue)
        key['response'] = pd.DataFrame(response)
        key['grace_period'] = pd.DataFrame(grace_period)
        key['wait_for_odor_l'] = pd.DataFrame(wait_for_odor_l)
        key['wait_for_odor_r'] = pd.DataFrame(wait_for_odor_r)
        key['odor_left'] = pd.DataFrame(odor_left)
        key['odor_right'] = pd.DataFrame(odor_right)
        key['reward_delay_left'] = pd.DataFrame(reward_delay_left)
        key['reward_delay_right'] = pd.DataFrame(reward_delay_right)
        key['left_port_check'] = pd.DataFrame(left_port_check)
        key['left_not_present'] = pd.DataFrame(left_not_present)
        key['right_port_check'] = pd.DataFrame(right_port_check)
        key['right_not_present'] = pd.DataFrame(right_not_present)
        key['left_big_reward'] = pd.DataFrame(left_big_reward)
        key['left_small_reward'] = pd.DataFrame(left_small_reward)
        key['right_big_reward'] = pd.DataFrame(right_big_reward)
        key['right_small_reward'] = pd.DataFrame(right_small_reward)
        key['outcome_delivery'] = pd.DataFrame(outcome_delivery)
        key['incorrect_left'] = pd.DataFrame(incorrect_left)
        key['incorrect_right'] = pd.DataFrame(incorrect_right)
        key['no_choice'] = pd.DataFrame(no_choice)
        key['incorrect'] = pd.DataFrame(incorrect)
        key['timeout_odor'] = pd.DataFrame(timeout_odor)
        key['timeout_reward_delay'] = pd.DataFrame(timeout_reward_delay)
        key['timeout_outcome'] = pd.DataFrame(timeout_outcome)
        key['intertrial_interval'] = pd.DataFrame(intertrial_interval)
        
        self.insert1(key)
        

TrialStates.populate()
TrialStates()

In [ ]:
@schema
class TrialEvents(dj.Imported):
    definition = """
    -> TrialData
    ---
    tup = null : blob
    port_1_in = null : blob
    port_1_out = null : blob
    port_2_in = null : blob
    port_2_out = null : blob
    port_3_in = null : blob
    port_3_out = null : blob
    global_timer_1_start = null : blob
    global_timer_1_end = null : blob
    global_timer_2_start = null : blob
    global_timer_2_end = null : blob
    global_timer_3_start = null : blob
    global_timer_3_end = null : blob
    global_timer_4_start = null : blob
    global_timer_4_end = null : blob
    global_counter_1_end = null : blob
    global_counter_2_end = null : blob
    global_counter_3_end = null : blob
    global_counter_4_end = null : blob
    """
    
    def make(self,key):
        try:
            tup = (TrialData & key).fetch1()['trial_events']['Tup'][0][0]
            key['tup'] = tup
        except KeyError:
            pass
        try:
            port_1_in = (TrialData & key).fetch1()['trial_events']['Port1In'][0][0]
            key['port_1_in'] = port_1_in
        except KeyError:
            pass
        try:
            port_1_out = (TrialData & key).fetch1()['trial_events']['Port1Out'][0][0]
            key['port_1_out'] = port_1_out
        except KeyError:
            pass
        try:
            port_2_in = (TrialData & key).fetch1()['trial_events']['Port2In'][0][0]
            key['port_2_in'] = port_2_in
        except KeyError:
            pass
        try:
            port_2_out = (TrialData & key).fetch1()['trial_events']['Port2Out'][0][0]
            key['port_2_out'] = port_2_out
        except KeyError:
            pass
        try:
            port_3_in = (TrialData & key).fetch1()['trial_events']['Port3In'][0][0]
            key['port_3_in'] = port_3_in
        except KeyError:
            pass
        try:
            port_3_out = (TrialData & key).fetch1()['trial_events']['Port3Out'][0][0]
            key['port_3_out'] = port_3_out
        except KeyError:
            pass
        try:
            global_timer_1_start = (TrialData & key).fetch1()['trial_events']['GlobalTimer1_Start'][0][0]
            key['global_timer_1_start'] = global_timer_1_start
        except KeyError:
            pass
        try:
            global_timer_1_end = (TrialData & key).fetch1()['trial_events']['GlobalTimer1_End'][0][0]
            key['global_timer_1_end'] = global_timer_1_end
        except KeyError:
            pass
        try:
            global_timer_2_start = (TrialData & key).fetch1()['trial_events']['GlobalTimer2_Start'][0][0]
            key['global_timer_2_start'] = global_timer_2_start
        except KeyError:
            pass
        try:
            global_timer_2_end = (TrialData & key).fetch1()['trial_events']['GlobalTimer2_End'][0][0]
            key['global_timer_2_end'] = global_timer_2_end
        except KeyError:
            pass
        try:
            global_timer_3_start = (TrialData & key).fetch1()['trial_events']['GlobalTimer3_Start'][0][0]
            key['global_timer_3_start'] = global_timer_3_start
        except KeyError:
            pass
        try:
            global_timer_3_end = (TrialData & key).fetch1()['trial_events']['GlobalTimer3_End'][0][0]
            key['global_timer_3_end'] = global_timer_3_end
        except KeyError:
            pass
        try:
            global_timer_4_start = (TrialData & key).fetch1()['trial_events']['GlobalTimer4_Start'][0][0]
            key['global_timer_4_start'] = global_timer_4_start
        except KeyError:
            pass
        try:
            global_timer_4_end = (TrialData & key).fetch1()['trial_events']['GlobalTimer4_End'][0][0]
            key['global_timer_4_end'] = global_timer_4_end
        except KeyError:
            pass
        try:
            global_counter_1_end = (TrialData & key).fetch1()['trial_events']['GlobalCounter1_End'][0][0]
            key['global_counter_1_end'] = global_counter_1_end
        except KeyError:
            pass
        try:
            global_counter_2_end = (TrialData & key).fetch1()['trial_events']['GlobalCounter2_End'][0][0]
            key['global_counter_2_end'] = global_counter_2_end
        except KeyError:
            pass
        try:
            global_counter_3_end = (TrialData & key).fetch1()['trial_events']['GlobalCounter3_End'][0][0]
            key['global_counter_3_end'] = global_counter_3_end
        except KeyError:
            pass
        try:
            global_counter_4_end = (TrialData & key).fetch1()['trial_events']['GlobalCounter4_End'][0][0]
            key['global_counter_4_end'] = global_counter_4_end
        except KeyError:
            pass
        
        self.insert1(key)

# TrialEvents.populate()
TrialEvents()


In [ ]:
@schema
class TrialProcessed(dj.Computed):
    definition = """
    -> TrialData
    -> TrialStates
    -> TrialEvents
    ---
    total_trial_time : float
    response_time = null : float
    trial_water : int
    outcome : int
    incorrect_trial : int
    forced_info_trial = 0 : int
    forced_random_trial = 0 : int
    no_choice_trial = 0 : int
    choice_trial = 0 : int
    choice_info = 0 : int
    choice_random = 0 : int
    big_reward = 0 : int
    small_reward = 0 : int
    """
    
    def make(self,key):
        
        dropVol = 4 #uL
        
        ## trial time
        total_trial_time = (TrialData & key).fetch1()['trial_end_timestamp'] - (TrialData & key).fetch1()['trial_start_timestamp']
        
        ## response time
        def None_sum(*args):
            args = [a for a in args if not a is None]
            return sum(args) if args else None

        response_time = None_sum((TrialStates & key).fetch1()['response'].iloc[1] - (TrialStates & key).fetch1()['go_cue'].iloc[1]) + ((TrialStates & key).fetch1()['grace_period'].iloc[1] - (TrialStates & key).fetch1()['grace_period'].iloc[0])[0]
        
        ## trial water
        try:
            trial_water = len((TrialEvents & key).fetch1()['global_timer_3_end'])
        except TypeError:
            try:
                trial_water = len((TrialEvents & key).fetch1()['global_timer_4_end'])
            except TypeError:
                trial_water = 0

        ## trial outcome
        outcome = (TrialData & key).fetch1()['outcome']
        
        ## error rate
        if np.isnan((TrialStates & key).fetch1()['incorrect_left'][0][0]) != True or np.isnan((TrialStates & key).fetch1()['incorrect_right'][0][0]) != True or np.isnan((TrialStates & key).fetch1()['incorrect'][0][0]) != True:
            key['incorrect_trial'] = 1
        else: 
            key['incorrect_trial'] = 0
            
        ## no choice
        if np.isnan((TrialStates & key).fetch1()['no_choice'][0][0]) != True:
            key['no_choice_trial'] = 1
        else:
            key['no_choice_trial'] = 0
        
        

        ##big/small reward
        if np.isnan((TrialStates & key).fetch1()['left_big_reward'][0][0]) != True or np.isnan((TrialStates & key).fetch1()['right_big_reward'][0][0]) != True:
            key['big_reward'] = 1
        elif np.isnan((TrialStates & key).fetch1()['left_small_reward'][0][0]) != True or np.isnan((TrialStates & key).fetch1()['right_small_reward'][0][0]) != True:
            key['small_reward'] = 1
        else: 
            pass

                
                
        ## trial types
        if (TrialData & key).fetch1()['trial_type'] == 1:
            key['choice_trial'] = 1
        elif (TrialData & key).fetch1()['trial_type'] == 2:
            key['forced_info_trial'] = 1
        elif (TrialData & key).fetch1()['trial_type'] == 3:
            key['forced_random_trial'] = 1
        
        
        ## choice trial random/info
        infoside = ((TrialData & key).fetch1()['trial_settings']['InfoSide'][0][0][0])
        if (TrialData & key).fetch1()['trial_type'] == 1:
            if infoside == 0:
                if np.isnan((TrialStates & key).fetch1()['OdorLeft'][0][0]) != True:
                    key['choice_info'] = 1
                else: 
                    key['choice_random'] = 1
            elif infoside == 1:
                if np.isnan((TrialStates & key).fetch1()['OdorLeft'][0][0]) != True:
                    key['choice_random'] = 1
                else:
                    key['choice_info'] = 1

#         key['choice_info'] = choice_info
#         key['choice_random'] = choice_random
        key['outcome'] = outcome
        key['total_trial_time'] = total_trial_time
        key['response_time'] = response_time[0]
        key['trial_water'] = trial_water * dropVol
        self.insert1(key)
        
TrialProcessed.populate()
TrialProcessed()


In [ ]:
# print(sum((TrialProcessed & 'mouse_id = "JB366"' & 'date = "2020-09-22"').fetch()['trial_water']))
# plt.plot(TrialProcessed.fetch()['response_time'])

In [ ]:
@schema
class PlotTable(dj.Computed):
    definition = """
    -> MouseAttributes
    
    ---
    num_trials_plot : longblob
    error_rate : longblob
    """
    
    def make(self,key):
        mouseID = np.unique((TrialProcessed).fetch()['mouse_id'])
        dates = np.unique((TrialProcessed).fetch()['date'])
        
        dayTrials = []
        errorRate = []
        for j in dates:
            dayTrials.append(len((TrialProcessed & key & 'date = ' + str('"' + str(j) + '"'))))
            
            
            #error rate

            randErrors = len(TrialProcessed & key & 'date = ' + str('"' + str(j) + '"') & 'forced_random_trial = "1"' & 'incorrect_trial = "1"')
            numRandTrials = len(TrialProcessed & key & 'date = ' + str('"' + str(j) + '"') & 'forced_random_trial = "1"')
            infoErrors = len(TrialProcessed & key & 'date = ' + str('"' + str(j) + '"') & 'forced_info_trial = "1"' & 'incorrect_trial = "1"')
            numInfoTrials = len(TrialProcessed & key & 'date = ' + str('"' + str(j) + '"') & 'forced_info_trial = "1"')
            
            errorRate.append(numErrors/numTrials)

                
                
        numTrialsPlot = plt.plot(dayTrials)
        plt.show()
        errorRatePlot = plt.plot(errorRate)
        plt.show()
        key['num_trials_plot'] = numTrialsPlot
        key['error_rate'] = errorRatePlot 
        self.insert1(key)
    
PlotTable.populate()
PlotTable()

In [ ]:
# # PlotTable.drop(True)
# @schema
# class PlotTable(dj.Computed):
#     definition = """
#     -> ExpDates
#     ---
#     testplot : longblob
#     """
    
#     def make(self,key):
#         plt.figure()
#         plt.title(key)
#         testplot = plt.plot((SessionData & key).fetch()['final_reward_delay'])
#         plt.ylim = (0,10)
#         key['testplot'] = testplot
#         print('plotTable', key['mouse_id'], ': populated')
#         self.insert1(key)
        
# dj.ERD(schema)

In [ ]:
# @schema
# class PerDaySummary(dj.Imported):
#     definition = """
#     -> DateTable
#     ---
#     day_total_water : int
#     error_rate : float
#     """
    
    
#     def make(self,key):
#         mouseID = (DateTable & key).fetch()['mouse_id']
#         dates = (DateTable & key).fetch()['date']
#         for i in mouseID:
#             for j in dates:
#                 dayWater = sum((SessionWater & 'mouse_id = ' + '"' + i + '"' & 'date = ' + '"' + str(j) + '"').fetch()['water_vol_ul'])
#                 try:
#                     error_rate = (len((TrialProcessed & 'incorrect_trial = "1"'& 'mouse_id = ' + '"' + i + '"' & 'date = ' + '"' + str(j) + '"')))/(len((TrialProcessed & 'incorrect_trial = "1"' & 'mouse_id = ' + '"' + i + '"' & 'date = ' + '"' + str(j) + '"')) + len((TrialProcessed & 'incorrect_trial = "0"' & 'mouse_id = ' + '"' + i + '"' & 'date = ' + '"' + str(j) + '"')))
#                 except ZeroDivisionError:
#                     error_rate = 0
#                 key['day_total_water'] = dayWater
#                 key['error_rate'] = error_rate
#         self.insert1(key)
                          
            
# PerDaySummary.populate()
# # (PerDaySummary & 'mouse_id = "JB366"').fetch()['outcomes'][0]
# PerDaySummary()
